In [ ]:
class WindFarm(Dataset):
    def __init__(self, data_dir, seq_len=1200):
        self.seq_len = seq_len
        # data = np.loadtxt(data_path, delimiter=',', skiprows=1, dtype=None)
        data = DataReader().get_results_data([data_dir])["lut_LUT"].to_pandas()
        
        wind_dir_cols = sorted([col for col in data.columns if "wind_direction" in col])
        wind_mag_cols = sorted([col for col in data.columns if "wind_speed" in col])
        turbine_yaw_cols = sorted([col for col in data.columns if "nacelle_direction" in col])

        self.X = self.normalize(data.loc[:, wind_dir_cols + wind_mag_cols + turbine_yaw_cols + turbine_power_cols].to_numpy()) # 5 variables
        self.y = data.loc[:, turbine_power_cols].to_numpy()
        self.num_labels = len(np.unique(self.y))
        self.len = len(self.y)

    def __len__(self):
        return (self.len - self.seq_len)-1

    def __getitem__(self, idx):
        x = np.transpose(self.X[idx:idx+self.seq_len])
        label = self.y[idx+self.seq_len+1]

        return torch.tensor(x, dtype=torch.float), torch.tensor(label, dtype=torch.float)
